In [0]:
#Implememntation based on: https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!unzip drive/My\ Drive/NLP/quora_dataset/test.csv.zip 
!unzip drive/My\ Drive/NLP/quora_dataset/train.csv.zip 

Archive:  drive/My Drive/NLP/quora_dataset/test.csv.zip
  inflating: test.csv                
Archive:  drive/My Drive/NLP/quora_dataset/train.csv.zip
  inflating: train.csv               


In [0]:
BASE_DIR = 'drive/My Drive/NLP/quora_dataset/'

In [0]:
import os
import csv
import codecs
import numpy as np
import pandas as pd
np.random.seed(1337)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, merge, LSTM, Lambda, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import sys

In [0]:
!ls

drive  sample_data  test.csv  train.csv


In [0]:
TRAIN_DATA_FILE = 'train.csv'
TEST_DATA_FILE = 'test.csv'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

In [0]:
print('Indexing word vectors.')
embeddings_index = {}
f = codecs.open(os.path.join(BASE_DIR, 'glove.6B.300d.txt'), encoding='utf-8')
for line in f:
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [0]:
#embeddings_index['unk'] = np.zeros((300,), dtype=float)
embeddings_index['ydahbd'] = np.zeros((300,), dtype=float)

In [0]:
questions = {}
f = codecs.open(os.path.join('drive/My Drive/NLP/quora_dataset/apple/', 'corpus.tsv'), encoding='utf-8')
for line in f:
  values = line.strip().split("\t")
  questions[values[0]] = values[1]

f.close()

source = []
f = codecs.open(os.path.join('drive/My Drive/NLP/quora_dataset/apple/', 'full.pos.txt'), encoding='utf-8')
i = 0
for line in f:
  source.append([])
  values = line.strip().split(" ")
  source[i].append('ydahbd '*30)
  source[i].append('ydahbd '*30)
  source[i].append(questions[values[0]])
  source[i].append(questions[values[1]])
  source[i].append(questions[values[0]])
  source[i].append(questions[values[1]])
  source[i].append(1)
  i+=1
f.close()

f = codecs.open(os.path.join('drive/My Drive/NLP/quora_dataset/apple/', 'full.neg.txt'), encoding='utf-8')
j = 0

for line in f:
  if j > 50000:
    break
  source.append([])
  values = line.strip().split(" ")
  source[i].append('ydahbd '*30)
  source[i].append('ydahbd '*30)
  source[i].append(questions[values[0]])
  source[i].append(questions[values[1]])
  source[i].append(questions[values[0]])
  source[i].append(questions[values[1]])
  source[i].append(0)
  i+=1
  j+=1

f.close()

data = source

In [0]:
source[7000]

['ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ',
 'ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ydahbd ',
 'PDF annotation tools for the iPad',
 'Reinstalling my Windows machine and wanting to preserve iTunes backups',
 'PDF annotation tools for the iPad',
 'Reinstalling my Windows machine and wanting to preserve iTunes backups',
 0]

In [0]:
np.random.seed(1)
np.random.shuffle(data)

In [0]:
train = np.array(data[5000:15000])
test = np.array(data[:5000])


In [0]:
del data, source

In [0]:
q1_train = list(train[:,0])
q2_train = list(train[:,1])
q3_train = list(train[:,2])
q4_train = list(train[:,3])
q5_train = list(train[:,4])
q6_train = list(train[:,5])
label_train = list(train[:,6])

q1_test = list(test[:,0])
q2_test = list(test[:,1])
q3_test = list(test[:,2])
q4_test = list(test[:,3])
q5_test = list(test[:,4])
q6_test = list(test[:,5])
label_test = list(test[:,6])

In [0]:
print('Processing text dataset')
texts_1 = [] 
texts_2 = []
labels = []  # list of label ids
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        q1_train.append(values[3])
        q2_train.append(values[4])
        q3_train.append(values[3])
        q4_train.append(values[4])
        q5_train.append('ydahbd '*30)
        q6_train.append('ydahbd '*30)
        label_train.append(int(values[5]))
print('Found %s texts.' % len(q1_train))

test_texts_1 = q1_test
test_texts_2 = q2_test
test_texts_3 = q3_test
test_texts_4 = q4_test
test_texts_5 = q5_test
test_texts_6 = q6_test
test_labels = label_test  # list of label ids

print('Found %s texts.' % len(test_texts_1))

Processing text dataset
Found 414290 texts.
Found 5000 texts.


In [0]:
len(q1_train)

414290

In [0]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(q1_train + q2_train + q3_train + q4_train)
sequences_1 = tokenizer.texts_to_sequences(q1_train)
sequences_2 = tokenizer.texts_to_sequences(q2_train)
sequences_3 = tokenizer.texts_to_sequences(q3_train)
sequences_4 = tokenizer.texts_to_sequences(q4_train)
sequences_5 = tokenizer.texts_to_sequences(q5_train)
sequences_6 = tokenizer.texts_to_sequences(q6_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

test_sequences_1 = tokenizer.texts_to_sequences(q1_test)
test_sequences_2 = tokenizer.texts_to_sequences(q2_test)
test_sequences_3 = tokenizer.texts_to_sequences(q3_test)
test_sequences_4 = tokenizer.texts_to_sequences(q4_test)
test_sequences_5 = tokenizer.texts_to_sequences(q5_test)
test_sequences_6 = tokenizer.texts_to_sequences(q6_test)


data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
data_3 = pad_sequences(sequences_3, maxlen=MAX_SEQUENCE_LENGTH)
data_4 = pad_sequences(sequences_4, maxlen=MAX_SEQUENCE_LENGTH)
data_5 = pad_sequences(sequences_5, maxlen=MAX_SEQUENCE_LENGTH)
data_6 = pad_sequences(sequences_6, maxlen=MAX_SEQUENCE_LENGTH)

labels = np.array(label_train)
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
test_data_3 = pad_sequences(test_sequences_3, maxlen=MAX_SEQUENCE_LENGTH)
test_data_4 = pad_sequences(test_sequences_4, maxlen=MAX_SEQUENCE_LENGTH)
test_data_5 = pad_sequences(test_sequences_5, maxlen=MAX_SEQUENCE_LENGTH)
test_data_6 = pad_sequences(test_sequences_6, maxlen=MAX_SEQUENCE_LENGTH)
test_labels = np.array(label_test)

del test_sequences_1
del test_sequences_2
del test_sequences_3
del test_sequences_4
del test_sequences_5
del test_sequences_6
del sequences_1
del sequences_2
del sequences_3
del sequences_4
del sequences_5
del sequences_6
import gc
gc.collect()

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 96729 unique tokens.
Shape of data tensor: (414290, 30)
Shape of label tensor: (414290,)


0

In [0]:
print('Preparing embedding matrix.')
# prepare embedding matrix
nb_words = min(MAX_NB_WORDS, len(word_index))+ 2

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix.
Null word embeddings: 36068


In [0]:
embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [0]:
num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

act = 'relu'
re_weight = True # whether to re-weight classes to fit the 17.5% share in test set

In [0]:
perm = np.random.permutation(len(data_1))
idx_train = perm[:int(len(data_1)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data_1)*(1-VALIDATION_SPLIT)):]

data_1_train = data_1[idx_train]
data_2_train = data_2[idx_train]
data_3_train = data_3[idx_train]
data_4_train = data_4[idx_train]
data_5_train = data_5[idx_train]
data_6_train = data_6[idx_train]

labels_train = labels[idx_train]

data_1_val = data_1[idx_val]
data_2_val = data_2[idx_val]
data_3_val = data_3[idx_val]
data_4_val = data_4[idx_val]
data_5_val = data_5[idx_val]
data_6_val = data_6[idx_val]

labels_val =labels[idx_val]

weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.472001959
    weight_val[labels_val==0] = 1.309028344

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [0]:
labels_train = np.array(labels_train).astype(int).tolist()
labels_val = np.array(labels_val).astype(int).tolist()

In [0]:
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, RNN, GRU
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [0]:
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
x2 = lstm_layer(embedded_sequences_2)

sequence_3_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_3_input)
x3 = lstm_layer(embedded_sequences_2)

sequence_4_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_4_input)
x4 = lstm_layer(embedded_sequences_2)

sequence_5_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_5_input)
x5= lstm_layer(embedded_sequences_2)

sequence_6_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_6_input)
x6 = lstm_layer(embedded_sequences_2)

merged = concatenate([x1, x2, x3,x4,x5,x6])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

In [0]:
if re_weight:
    class_weight = {0: 1.309028344, 1: 0.472001959}
else:
    class_weight = None

In [0]:
model = Model(inputs=[sequence_1_input, sequence_2_input,sequence_3_input,sequence_4_input,sequence_5_input,sequence_6_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc',get_f1])

In [0]:
STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

In [0]:
#model.summary()
print(STAMP)

early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit([data_1_train, data_2_train, data_3_train, data_4_train, data_5_train, data_6_train], labels_train, \
        validation_data=([data_1_val, data_2_val, data_3_val,data_4_val,data_5_val,data_6_val], labels_val, weight_val), \
        epochs=200, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_loss'])

lstm_225_133_0.19_0.27
Train on 372861 samples, validate on 41429 samples
Epoch 1/200
372861/372861 [==============================] - 110s 294us/step - loss: 0.4355 - acc: 0.6897 - get_f1: 0.3917 - val_loss: 0.3075 - val_acc: 0.6560 - val_get_f1: 0.1109
Epoch 2/200
372861/372861 [==============================] - 107s 286us/step - loss: 0.3460 - acc: 0.7233 - get_f1: 0.4277 - val_loss: 0.2967 - val_acc: 0.6735 - val_get_f1: 0.1981
Epoch 3/200
372861/372861 [==============================] - 107s 286us/step - loss: 0.3231 - acc: 0.7419 - get_f1: 0.4870 - val_loss: 0.2619 - val_acc: 0.7125 - val_get_f1: 0.3751
Epoch 4/200
372861/372861 [==============================] - 106s 285us/step - loss: 0.3071 - acc: 0.7555 - get_f1: 0.5290 - val_loss: 0.2176 - val_acc: 0.7646 - val_get_f1: 0.6001
Epoch 5/200
372861/372861 [==============================] - 106s 283us/step - loss: 0.2931 - acc: 0.7691 - get_f1: 0.5678 - val_loss: 0.2107 - val_acc: 0.7886 - val_get_f1: 0.6354
Epoch 6/200
372861/37

In [0]:
preds = model.predict([test_data_1, test_data_2, test_data_3, test_data_4, test_data_5, test_data_6], batch_size=500, verbose=1)



5000/5000 [==============================] - 1s 199us/step


In [0]:
new_preds = list(preds)
to_check = []
print(new_preds[1])
for i in range(len(new_preds)):
  if new_preds[i][0] > 0.05:
    to_check.append(1)
  else:
    to_check.append(0)

[2.8539635e-06]


In [0]:
new_labels = np.array(test_labels).astype(int).tolist()
acc = 0
for i in range(len(to_check)):
  if to_check[i] == new_labels[i]:
    acc +=1
acc = acc/ len(to_check)

In [0]:
acc

0.943

In [0]:
# 0.73346 #1000 training data ---> Run all Siamese
from sklearn.metrics import classification_report

y_test = new_labels
#y_pred_bool = np.argmax(new_preds, axis=1)
y_pred_bool = to_check

print(classification_report(y_test, y_pred_bool))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4749
           1       0.43      0.42      0.42       251

    accuracy                           0.94      5000
   macro avg       0.70      0.69      0.70      5000
weighted avg       0.94      0.94      0.94      5000



In [0]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred_bool)

0.6945286866851399